<img src="https://imgur.com/asJQHOg.png">

# Table of Contents:

**1. [Introduction](#Introduction)** <br>
**2. [Import libraries](#Libraries)** <br>
**3. [Knowning the data](#Data)** <br>
**4. [EDA](#Explorations)** <br>
**5. [Preprocessing](#Prepocess)** <br>
**6. [Model Training](#Model)** <br>
**7. [Validation](#Validation)** <br>

<a id="Introduction"></a> <br> 
# **1. Introduction:** 
Although the data used for this competition is synthetic, it is based on a real dataset and generated using a CTGAN. The original dataset deals with predicting the category on an eCommerce product given various attributes about the listing. 

> 🎯 Goal: To predict the probability the id belongs to each class

> 📖 Data:
> - ```train.csv``` - *training data*, one product (id) per row, with the associated features (feature_*) and class label (target)
> - ```test.csv``` - *test data*

<a id="Libraries"></a> <br> 
# **2. Import libraries 📚** 

In [ ]:
#This librarys is to work with matrices
import pandas as pd 
# This librarys is to work with vectors
import numpy as np
# This library is to create some graphics algorithmn
import seaborn as sns
# to render the graphs
import matplotlib.pyplot as plt
#This library use for building ANN model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras
from tensorflow.keras.utils import to_categorical
from keras.layers import Input
#This library use for data preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# This function makes the plot directly on browser
%matplotlib inline

<a id="Data"></a> <br> 
# **3. First look at the data:** 

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
#Looking data format and types
print(train.info())

In [ ]:
# printing test info()
print(test.info())

The ``id`` value is meaningless, so I will leave it out in advance.

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

<a id="Explorations"></a> <br> 
# **4. EDA 📊** 

In [ ]:
fig, ax = plt.subplots()
sns.countplot(x='target', data=train, order=sorted(train['target'].unique()), ax=ax)
ax.set_ylim(0, 63000)
ax.set_title('Target Distribution', weight='bold')
plt.show()

First, let's look at the statistical values for each feature.

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

**Summary**
* No missing data

* The representative statistics of train and test are almost similar.

* There is no significant difference between the mean and the deviation.

* There are cases where the minimum value and the maximum value are different, which means that the feature range of the test may be different in the train.

<a id="Preprocess"></a> <br> 
# **5. Preprocessing :** 

**Train-Test Split & Normalization 🏷️**

The data needs to be normalized and split train-test to fit into the ANN

In [ ]:
for i in range(50):
    mean, std = train[f'feature_{i}'].mean(), train[f'feature_{i}'].std()
    train[f'feature_{i}'] = train[f'feature_{i}'].apply(lambda x : (x-mean)/std)
    test[f'feature_{i}'] = test[f'feature_{i}'].apply(lambda x : (x-mean)/std)

In [ ]:
label = {var:index for index, var in enumerate(sorted(train['target'].unique()))}
train['target'] = train['target'].map(label)

target = train['target']
train.drop(['target'], inplace=True, axis=1)

In [ ]:
train = train.values
target = target.values
target =  to_categorical(target)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size = 0.1, random_state = 2, stratify=target)

<a id="Model"></a> <br> 
# **6. Model Build and Training ⚙️** 

**Theory of ANN**

An artificial neural network is a supervised learning algorithm which means that we provide it the input data containing the independent variables and the output data that contains the dependent variable. For instance, in our example our independent variables are X1, X2 and X3. The dependent variable is Y.

In the beginning, the ANN makes some random predictions, these predictions are compared with the correct output and the error(the difference between the predicted values and the actual values) is calculated. The function that finds the difference between the actual value and the propagated values is called the cost function. The cost here refers to the error. Our objective is to minimize the cost function. Training a neural network basically refers to minimizing the cost function. We will see how we can perform this task.

A neural network executes in two phases: **Feed Forward phase** and **Back Propagation phase**. Let us discuss both these steps in detail.

**Feed Forward**

In the feed-forward phase of ANN, predictions are made based on the values in the input nodes and the weights. If you look at the neural network in the above figure, you will see that we have three features in the dataset: X1, X2, and X3, therefore we have three nodes in the first layer, also known as the input layer.

The weights of a neural network are basically the strings that we have to adjust in order to be able to correctly predict our output. For now, just remember that for each input feature, we have one weight.

<img src ="https://imgur.com/mninr5z.png">

**Back Propagation**

In the beginning, before you do any training, the neural network makes random predictions which are of course incorrect.
We start by letting the network make random output predictions. We then compare the predicted output of the neural network with the actual output. Next, we update the weights and the bias in such a manner that our predicted output comes closer to the actual output. In this phase, we train our algorithm.

<img src = "https://imgur.com/FI0R6lf.png" width="1080" height="1080">

**How do Neural networks learn?**

Looking at an analogy may be useful in understanding the mechanisms of a neural network. Learning in a neural network is closely related to how we learn in our regular lives and activities — we perform an action and are either accepted or corrected by a trainer or coach to understand how to get better at a certain task. Similarly, neural networks require a trainer in order to describe what should have been produced as a response to the input. Based on the difference between the actual value and the predicted value, an error value also called **Cost Function** is computed and sent back through the system.

<img src = "https://imgur.com/FQ2Tabx.gif" width="1080" height="1080">

In [ ]:
# Creating the model
model = Sequential()

# Inputing the first layer with input dimensions
model.add(Dense(32,activation='relu',input_dim=50,kernel_initializer='uniform'))
#The argument being passed to each Dense layer (32) is the number of hidden units of the layer. 
# A hidden unit is a dimension in the representation space of the layer.


# Adding an Dropout layer to previne from overfitting
model.add(Dropout(0.3))

#adding second hidden layer 
model.add(Dense(64,kernel_initializer='uniform',activation='relu'))

# Adding another Dropout layer
model.add(Dropout(0.2))

#adding third hidden layer 
model.add(Dense(128,kernel_initializer='uniform',activation='relu'))

# Adding another Dropout layer
model.add(Dropout(0.2))


# adding the output layer that is categorical
model.add(Dense(4,kernel_initializer='uniform',activation='softmax'))
#With such a scalar sigmoid output on a categorical classification problem, the loss
#function you should use is categorical_crossentropy

#Visualizing the model
model.summary()

Finally, we need to choose a loss function and an optimizer.

In [ ]:
# Compiling our model
model.compile(optimizer = 'sgd', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])
#optimizers list
#optimizers['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

# Fitting the ANN to the Training set
model.fit(X_train, y_train, epochs = 30, verbose=2)

<a id="Validation"></a> <br> 
# **7. Validation** 

In [ ]:
# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                    epochs=100, verbose=0)

# list all data in history
print(history.history.keys())

**Evaluating the model**

In [ ]:
scores = model.evaluate(X_train, y_train, batch_size=30)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# summarizing historical accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

<a id="Submission"></a> <br> 
# **8. Make Submission file 📝** 

In [ ]:
sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4']] = model.predict(test)
sample_submission.to_csv(f'tps_ann.csv',index=False)
sample_submission

If the content is helpful, please upvote. 🙂 Work in progress 🚧

## References:
1. [TPS May: RAPIDS 🏃‍](https://www.kaggle.com/ruchi798/tps-may-rapids)
2. [Build an Artificial Neural Network From Scratch: Part 1](https://www.kdnuggets.com/2019/11/build-artificial-neural-network-scratch-part-1.html)
3. [Titanic [EDA] + Model Pipeline + Keras NN](https://www.kaggle.com/kabure/titanic-eda-model-pipeline-keras-nn)
4. [[TPS-May] Categorical EDA](https://www.kaggle.com/subinium/tps-may-categorical-eda)